## Import 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from kite_trade import *
# import ta

import talib as ta
from tqdm import tqdm
import datetime
import pyarrow.parquet as pq
import pyarrow as pa

import multiprocessing
import time
import defs
from tqdm import tqdm

from pandas.tseries.offsets import MonthEnd
from datetime import timedelta

import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import mplfinance as mpf

import util_fun as uf

In [4]:
# import pywhatkit
# import datetime
# for i in range(1,3):
#     now = datetime.datetime.now()
#     pywhatkit.sendwhatmsg('+919368327140', 'I LOVE YOU', now.hour,now.minute+1,15,True,2)

In [5]:
enctoken = "xkgpBYvRSbNnKZCurQICbzKDuNEVay7vpidjrEhIre+T6O5yPAy1RCvBRrbD03wPK3YU7g82Tv6GMnZ+5gHm/22xMj3GLHkzIQpHDUSAt/HlfnImc7ArsA=="
kite = KiteApp(enctoken=enctoken)

## Data Fetch 

In [6]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
inst_filter.query("Symbol == 'NIFTY BANK'")

In [10]:
import time
start_date = '2023-01-01'
end_data = '2023-01-19'
symbol = 'NIFTY BANK'
signal_candle_low = None
import datetime
while True:
    if datetime.datetime.now().minute % 5 == 4:
        df_min_5 = uf.get_instrument_data(
            kite,
            inst_filter,
            timerframe_list = timerframe_list[3:4],
            start_date =start_date ,
            end_date = end_data,
            symbol = symbol)
        df_min_5.drop(df_min_5.tail(1).index,inplace=True)

        df_min_5_sev = df_min_5[df_min_5.index.strftime('%Y-%m-%d').isin(["2023-01-18","2023-01-19"]) ].copy()
        df_min_5_sev['ema_5'] = ta.EMA(df_min_5_sev['Close'], timeperiod = 5)
        df_min_5_sev.dropna(inplace=True)
        df_min_5_sev['above_5_ema'] = df_min_5_sev['Low'] > df_min_5_sev['ema_5']

        print(df_min_5_sev.tail(10))
        print(df_min_5_sev['above_5_ema'][-1],df_min_5_sev['Low'][-1])
        if df_min_5_sev['above_5_ema'][-1] == True:
            # store the time and low of price 
            #   for next candle check it if actually above ema 
            signal_candle_low = df_min_5_sev['Low'][-1]
            t_end = time.time() + 60 * 5
            while time.time() < t_end:
                ltp = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
                if ltp < signal_candle_low:
                    print(ltp)
                    print("buy condition meet")
                    print("sl")
                    # print(max(df_min_5_sev['High'][-1:-3]))
                time.sleep(10)
                   
        else:
            ## next candle closes neither above 5 ema and nor below previous low
            if signal_candle_low == None:
                time.sleep(300)
            else:
                t_end = time.time() + 60 * 5
                while time.time() < t_end:
                    ltp = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
                    if ltp < signal_candle_low:
                        print(ltp)
                        print("buy condition meet")
                        print("sl")
                        # print(max(df_min_5_sev['High'][-1:-3]))
                    time.sleep(10)

2023-01-01 2023-01-19
5minute


100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


                               Open      High       Low     Close  \
Date                                                                
2023-01-19 18:50:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 18:55:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:00:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:05:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:10:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:15:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:20:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:25:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:30:00+05:30  42328.85  42328.85  42328.85  42328.85   
2023-01-19 19:35:00+05:30  42328.85  42328.85  42328.85  42328.85   

                                  ema_5  above_5_ema  
Date                                                  
2023-01-19 18:50:00+05:30  42328.849999         True  
2023-0

In [ ]:

# watch for ellate candle -> till the low of candle is above 5 ema ( or take first ellate candle low breakout)
# entry: candle break ellate candle low
# SL: above elate candle high or 10-30rs or last2 or 3 candle high or swing high
# target: 1:3 or 1:2

## findings 
# seconds it take to update 5 min candles -> its updating in between also 
# order place try



# from ta import add_all_ta_features
# from ta.utils import dropna

## order place

In [ ]:
# kite.orders()

In [ ]:
# order = kite.place_order(
#     variety = kite.VARIETY_REGULAR,
#     exchange = kite.EXCHANGE_NFO,
#     tradingsymbol = "BANKNIFTY2311942500PE",
#     transaction_type =kite.TRANSACTION_TYPE_SELL,
#     quantity = 25,
#     product = kite.PRODUCT_MIS,
#     order_type = kite.ORDER_TYPE_MARKET,
#     price=None,
#     validity=kite.VALIDITY_DAY,
#     disclosed_quantity=None,
#     trigger_price=None,
#     squareoff=None,
#     stoploss=None,
#     trailing_stoploss=None,
#     # tag="TradeviaPython",
# )


## pivot

In [ ]:
#CPR
# Pivot = (High + Low + Close)/3
# Bottom CPR = (High + Low)/ 2
# Top CPR = (Pivot – BC) + Pivot

#  open  previous day open cpr
# is opening 
# 1.gap up or gap down 
# 2. is open above or below  PL or PH  -> done
# 3. wide cpr or narrow cpr -> done
# 4. break the cpr in between the day 
# 5. 

# next step -> look for video of
# 1. options
# 2. 25 stretagy and code them 

# ->other thiing i am thinking write a breakdown code in last 15 days 
# -> data dump in sql lite 

In [ ]:
df_day_fetch = temp1.copy()
df_min_5 = temp2.copy()
# %chg
df_day_fetch_temp = df_day_fetch.copy()
df_day_fetch_temp['pct_chng'] = df_day_fetch_temp['Close'].pct_change() * 100

# pivot
df_day_fetch['pivot'] = (df_day_fetch['High'] + df_day_fetch['Low'] + df_day_fetch['Close'])/3
df_day_fetch['bottom_pivot'] =  (df_day_fetch['High'] + df_day_fetch['Low'])/2
df_day_fetch['top_pivot'] = (df_day_fetch['pivot'] - df_day_fetch['bottom_pivot']) + df_day_fetch['pivot']
df_day_fetch = df_day_fetch.shift(1)

# adding preiouse day price 
df_day_fetch.rename(
    columns = {'Close': "prev_close", 'High':"prev_high","Low": "prev_low",
     "Open":'prev_open'}, inplace = True)
df_day_fetch = df_day_fetch_temp.merge(df_day_fetch,left_index=True, right_index=True)


df_min_5['date_only'] = pd.to_datetime(df_min_5.index.date)
# df_min_5_temp = df_min_5[df_min_5.date_only ==pd.to_datetime('2023-01-13')].copy()
df_min_5_temp = df_min_5.copy()
df_day_fetch['date_only'] = pd.to_datetime(df_day_fetch.index.date)
df_min_5_temp.reset_index(inplace = True)
df_min_5_temp = df_min_5_temp.merge(df_day_fetch,how = 'left', on = 'date_only',suffixes=('_min','_day'))
df_min_5_temp.dropna(inplace =True)

In [ ]:
def gap_up_down_open(df):
    if (df['Date'].strftime("%H:%M:%S") == '09:15:00')  &   (df['Open_min'] >= df['prev_high']):
        return 'up'
    elif (df['Date'].strftime("%H:%M:%S") ==pd.to_datetime('09:15:00'))  &   (df['Open_min'] >= df['prev_low']):
        print(df['Date'])
        return 'down'
    
df_min_5_temp['gap_up_down'] = df_min_5_temp.apply(lambda x: gap_up_down_open(x), axis= 1)  

In [ ]:
# getting narrow and wide cpr
df_min_5_temp['close_1per'] = df_min_5_temp['prev_close'] * .001
df_min_5_temp['tc-bc'] = abs(df_min_5_temp['top_pivot'] - df_min_5_temp['bottom_pivot'])
df_min_5_temp['cpr_range']  = np.where(df_min_5_temp['tc-bc'] < df_min_5_temp['close_1per'], "narrow", 'wide')

In [ ]:
# df_min_5_temp.tail(50)

In [ ]:
df_day_fetch['pivot'] = (df_day_fetch['High'] + df_day_fetch['Low'] + df_day_fetch['Close'])/3
df_day_fetch['bottom_pivot'] =  (df_day_fetch['High'] + df_day_fetch['Low'])/2
df_day_fetch['top_pivot'] = (df_day_fetch['pivot'] - df_day_fetch['bottom_pivot']) + df_day_fetch['pivot']

In [ ]:
# df_min_5_temp[['date_only','pivot','cpr_range']].drop_duplicates().tail(50)

In [ ]:
# df_min_5_temp[df_min_5_temp['gap_up_down'] == 'up'][['Date','pct_chng']].tail(50)

## Plots

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df_day_fetch.index,
                open=df_day_fetch['Open'],
                high=df_day_fetch['High'],
                low=df_day_fetch['Low'],
                close=df_day_fetch['Close'])])

fig.update_layout(
    width=1200, height=600,
    title="Bank nifty",
    yaxis_title='Bank',
    xaxis_rangeslider_visible=False
)
fig.show()